In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from accelerate import Accelerator
import os

In [ ]:
os.environ['HF_TOKEN']=""
model_id = "meta-llama/Llama-3.2-1B-Instruct"

In [4]:
# Accelerator 초기화
accelerator = Accelerator()

# 모델과 토크나이저 로딩
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    use_auth_token=True
)

# model, tokenizer를 accelerator 준비 상태로 변환
model, tokenizer = accelerator.prepare(model, tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:809: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [5]:
# pipeline 생성
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [13]:
# 입력 파일 및 대상 인물 설정
input_files = [
    "/content/drive/MyDrive/인공지능기초/harris_youtube_comment_filtered_for_llm_downsampled.jsonl",
    "/content/drive/MyDrive/인공지능기초/trump_youtube_comment_filtered_for_llm_downsampled.jsonl"
]
target_persons = ["Harris", "Trump"]

# 출력 파일 경로 설정
output_file = "/content/drive/MyDrive/인공지능기초/llm_outputs.jsonl"

# 진행 상황 저장용 파일
progress_file = output_file + ".progress"

In [15]:
# 기존 진행 상황 로드
if os.path.exists(progress_file):
    with open(progress_file, "r") as pf:
        processed_indices = json.load(pf)
else:
    processed_indices = {}  # {input_file: last_processed_index}


In [16]:
# N개 단위 저장 기준
save_interval = 10

In [ ]:
for input_file, target_person in zip(input_files, target_persons):
    # 파일별 처리 시작 인덱스 확인
    start_idx = processed_indices.get(input_file, 0)

    # 댓글 로드
    with open(input_file, "r") as f:
        comments = [json.loads(line)["textDisplay"] for line in f]

    # 이미 처리한 인덱스를 건너뛰기
    comments_to_process = comments[start_idx:]

    outputs = []

    for i, comment in enumerate(comments_to_process, start=start_idx):
        messages = [
            {"role": "system", "content": """You are an expert sentiment analyst who scores the sentiment and relevance to specific people (harris or trump) of comments."""},
            {"role": "user", "content": f"""
*Task:
1. Analyze the given comment and score its relevance to two people (harris, trump) with two scores (harris score, trump score).
2. Also, score the overall sentiment of the comment with three metrics (positive score, neutrality score, negative score).

*Comment:
- Analyze this comment: {comment}

*Context:
- Each score must be a float between 0 and 1
- The scores for each person (harris score, trump score) must be different.
- Assume the comment is from a YouTube video related to {target_person}'s election campaign.
- Do not include any text other than the output format.

*Output Format:
{{
  "harris score":,
  "trump score":,
  "positive score":,
  "neutrality score":,
  "negative score":
}}
            """},
        ]

        # 모델 출력
        output = pipe(
            messages,
            max_new_tokens=100,
            do_sample=True,
            temperature=0.7
        )

        generated_text = output[0]["generated_text"]
        outputs.append({"comment": comment, "output": generated_text})

        # N개(여기서는 5개) 마다 저장
        if (i+1) % save_interval == 0:
            # 결과 저장
            with open(output_file, "a") as f:
                for o in outputs:
                    json.dump(o, f, ensure_ascii=False)
                    f.write("\n")
            outputs = []

            # 진행 상황 저장
            processed_indices[input_file] = i+1
            with open(progress_file, "w") as pf:
                json.dump(processed_indices, pf)

            print(f"Processed {i+1} comments for {target_person} (intermediate save)")

    # 남은 결과 저장
    if outputs:
        with open(output_file, "a") as f:
            for o in outputs:
                json.dump(o, f, ensure_ascii=False)
                f.write("\n")
        outputs = []

        # 진행 상황 저장
        processed_indices[input_file] = i+1
        with open(progress_file, "w") as pf:
            json.dump(processed_indices, pf)

    print(f"Finished processing {input_file}")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Processed 8320 comments for Harris (intermediate save)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Processed 8330 comments for Harris (intermediate save)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

Processed 7520 comments for Trump (intermediate save)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Processed 7530 comments for Trump (intermediate save)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Processed 7540 comments for Trump (intermediate save)
